In [11]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

from model import DOFENTrainer
from dofen_default_config import dofen_config, train_config, eval_config

In [ ]:
# 1. Load data
df = pd.read_csv('index_1.csv')

# 2. Data preprocessing

# --- Feature Engineering: Create time-based numerical features ---
# Convert 'datetime' column to the proper datetime object
df['datetime'] = pd.to_datetime(df['datetime'])

df['hour'] = df['datetime'].dt.hour
df['day_of_week'] = df['datetime'].dt.dayofweek # Monday=0, Sunday=6
df['month'] = df['datetime'].dt.month
# Drop the original date/time columns
df = df.drop(columns=['date', 'datetime'])


# --- Handle Missing Values and Define Features ---
# The 'card' column has missing values. Fill them with a distinct category.
df['card'] = df['card'].fillna('ANON-MISSING')

# Define features
numerical_features = ['hour', 'day_of_week', 'month']
categorical_features = ['cash_type', 'card', 'coffee_name']
target_variable = 'money' # Regression target

# Separate features (X) and target (Y)
# We do not drop rows, as we imputed the only missing categorical column ('card').


# --- Encode Categorical Features (Label Encoding for DOFEN) ---
label_encoders = {}
category_counts = []

for col in categorical_features:
    le = LabelEncoder()
    # Fit and transform the categorical column
    df[col + '_encoded'] = le.fit_transform(df[col].astype(str))
    label_encoders[col] = le
    category_counts.append(len(le.classes_))

# Prepare feature matrix (categorical first, then numerical)
encoded_cat_cols = [col + '_encoded' for col in categorical_features]
x = df[encoded_cat_cols + numerical_features].values.astype('float32')
y = df[target_variable].values.astype('float32')


# --- Normalize the Numerical Features ---
scaler = StandardScaler()
# The numerical features start at the index right after the categorical ones
numerical_start_index = len(category_counts)
x[:, numerical_start_index:] = scaler.fit_transform(x[:, numerical_start_index:])


# 3. Train / test split
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42
)

tr_x, tr_y = x_train, y_train
te_x, te_y = x_test, y_test


# 4. DOFEN config update
# Update DOFEN config with the new column structure
# The categorical features are first, followed by the numerical features (which we treat as continuous)
dofen_config['column_category_count'] = category_counts + [-1] * len(numerical_features)
dofen_config['n_class'] = -1 # -1 for regression (predicting continuous 'money' values)

# Ensure your improved hyperparameters are still used:
dofen_config['m'] = 32
dofen_config['d'] = 5
dofen_config['n_forest'] = 80
dofen_config['n_hidden'] = 256
dofen_config['dropout'] = 0.35
train_config['n_epochs'] = 250
train_config['batch_size'] = 128

# 5. Trainer
dofen_trainer = DOFENTrainer(dofen_config, train_config, eval_config)
dofen_trainer.init()

# 6. Fit and evaluate
dofen_trainer.fit(tr_x, tr_y)
dofen_trainer.evaluate(te_x, te_y)

total condition:  960
n_rodt:  192
n_estimator:  64
Epoch: 1
Epoch: 2
Epoch: 3
Epoch: 4
Epoch: 5
Epoch: 6
Epoch: 7
Epoch: 8
Epoch: 9
Epoch: 10
Epoch: 11
Epoch: 12
Epoch: 13
Epoch: 14
Epoch: 15
Epoch: 16
Epoch: 17
Epoch: 18
Epoch: 19
Epoch: 20
Epoch: 21
Epoch: 22
Epoch: 23
Epoch: 24
Epoch: 25
Epoch: 26
Epoch: 27
Epoch: 28
Epoch: 29
Epoch: 30
Epoch: 31
Epoch: 32
Epoch: 33
Epoch: 34
Epoch: 35
Epoch: 36
Epoch: 37
Epoch: 38
Epoch: 39
Epoch: 40
Epoch: 41
Epoch: 42
Epoch: 43
Epoch: 44
Epoch: 45
Epoch: 46
Epoch: 47
Epoch: 48
Epoch: 49
Epoch: 50
Epoch: 51
Epoch: 52
Epoch: 53
Epoch: 54
Epoch: 55
Epoch: 56
Epoch: 57
Epoch: 58
Epoch: 59
Epoch: 60
Epoch: 61
Epoch: 62
Epoch: 63
Epoch: 64
Epoch: 65
Epoch: 66
Epoch: 67
Epoch: 68
Epoch: 69
Epoch: 70
Epoch: 71
Epoch: 72
Epoch: 73
Epoch: 74
Epoch: 75
Epoch: 76
Epoch: 77
Epoch: 78
Epoch: 79
Epoch: 80
Epoch: 81
Epoch: 82
Epoch: 83
Epoch: 84
Epoch: 85
Epoch: 86
Epoch: 87
Epoch: 88
Epoch: 89
Epoch: 90
Epoch: 91
Epoch: 92
Epoch: 93
Epoch: 94
Epoch: 95
Epoch: 